### **Bioprospecting pipeline 1.0**

***
#### **Introduction**
***

A Biosynthetic Gene Cluster (BGC) can be defined as a physically clustered group of two or more genes that encode the biosynthetic enzymes of a pathway (Medema 2015). The compounds encoded by BGCs represent diverse chemical structural classes, which include among others, polyketides, peptides, oligosaccharides, terpenoids, and alkaloids. These are commonly known as natural products and are a rich source of molecule candidates for the development of pharmaceutical applications, for instance, as vitamins, antibiotics, and antifungals (Newman and Cragg 2020). In addition, from an ecological perspective, natural  product BGCs are traits with key roles for the organisms’ survival and adaptation to the environment, participating in defense, nutrient scavenging, quorum sensing, etc (Fischbach and Voigt 2010; O’Brien and Wright 2011).  

The advent of new sequencing technologies and the availability of large numbers of genome sequences has boosted our capacity to mine encoded BGCs, especially from microorganisms, which are known to be highly prolific producers of natural products. As a result, genome mining has accelerated the discovery of new BGCs, positioning as a central approach to study these genomic elements (Ziemert, Alanjaryab, and Weber 2016). Metagenomics, as a culture-independent approach, offers unique opportunities to mine the natural product BGCs encoded by environmental microbial communities (Reddy 2012; Charlop-Powers 2015; Lemetre 2017; Rego 2020). However, given the long nature of these genomic elements, commonly in the order of tens of kb, their identification in metagenomic data requires exceptionally high-quality assemblies. In spite of recent advances (e.g., biosyntheticSPAdes (Meleshko 2019) and BiG-MAP (Pascal Andreu 2021)), identifying the different BGC classes in metagenomic data represents a major challenge that must be addressed to be able to expand bioprospecting endeavors (Nayfach 2021; Robinson 2021; Paoli 2022).  

The workflow we will develop will capitalize on recent bioinformatic advances in BGC bioprospecting analysis to maximize the exploitation of metagenomic data. Briefly, our strategy will consist of generating metagenome assemblies, including the reconstruction of metagenome assembled genomes (MAGs), identifying BGC sequences, clustering these sequences into Gene Cluster Families (GCFs), computing the BGC coverage and GCFs abundance, and performing the functinoal and taxonomic annotation.
In Fig. 1 we represent the implementation of this workflow.

![Figure 1](../figures/Bioprospectig_reads_vs_assembly_dev.png)

**Fig. 1**. Assembly-based bioprospecting pipeline. The input data consists of metagenomic samples previously preprocessed and assembled utilizing [VEBA](https://github.com/jolespin/veba). The pipeline is organized in the following tasks. **1) Identify BGC sequences:** the BGCs are annotated in the assembled metagenomic data utilizing [antiSMASH](https://github.com/antismash/antismash); **2) Taxonomic annotation:** the metagenomic contigs containing BGC sequences are taxonomically annotated utilizing [MMseqs taxonomy](https://github.com/soedinglab/MMseqs2#taxonomy) and the reference database [UniRef100](https://www.uniprot.org/help/uniref); **3) BGC mapping:** the metagenomic BGC sequences are mapped against (previously constructed) Gene Cluster Family (GCF) models of the MIBiG database v3 [MIBiG v3](https://mibig.secondarymetabolites.org/) utilizing [BiG-SLICE](https://github.com/pereiramemo/bigslice); **4) BGC clustering:** the metagenomic BGC sequences are clustered into GCFs with [BiG-SLICE](https://github.com/pereiramemo/bigslice); **5) Compute coverage:** the coverage of the metagenomic contigs, previously computed with [VEBA](https://github.com/jolespin/veba), is utilized to determine the coverage of the BGC sequences. The output of the pipeline consists of the following tables: 1) The metadata table containing the functional and taxonomic annotation, and the biosynthetic novelty; 2) The BGC class abundance table; 3) The GCFs abundance tables.

***
#### **Prepare the work environment**
***

In [2]:
%load_ext rpy2.ipython
%set_env WORKDIR=workdir
%set_env REPO=/home/epereira/workspace/dev/new_atlantis/repos/bioprospecting
%set_env seqtk=/nfs/bin/seqtk/seqtk

env: WORKDIR=workdir
env: REPO=/home/epereira/workspace/dev/new_atlantis/repos/bioprospecting
env: seqtk=/nfs/bin/seqtk/seqtk


In [ ]:
%%bash
mkdir -p ${WORKDIR}/data/sola
mkdir -p ${WORKDIR}/outputs/antismash/taxonomy
mkdir ${WORKDIR}/outputs/bgc_abund
mkdir ${WORKDIR}/outputs/bgc_taxa
mkdir ${WORKDIR}/outputs/tables


***
#### **Input data**
***

The input data, as described in the figure above, are metagenomic samples previously preprocessed and assembled with VEBA. For the purpose of this notebook, we will analyze 38 metagenomes of the [SOLA dataset](https://pubmed.ncbi.nlm.nih.gov/29925880/). Namenly, the input consists of assembled contigs and the mapping information (i.e., bam files).

We can download the data from the New Atlantis Cloud Lab.  

In [ ]:
%%bash

# aws s3 cp s3://newatlantis-case-studies/SOLA-samples/ ${WORKDIR}/data/sola --recursive
# aws s3 cp s3://newatlantis-case-studies/mibig_gcf_models/ ${WORKDIR}/data/mibig_gcf_models --recursive


***
#### **1) Identify BGC sequences**
***

For this we will be using our wrap script [run_antismash](https://github.com/pereiramemo/bioprospecting/blob/main/run_scripts/run_antismash.sh), which runs a our container of antiSMASH.  

In [ ]:
%%bash

SCAFFOLDS=$(ls ${WORKDIR}/data/sola/ERR*/output/scaffolds.fasta)
for SCAFFOLD in ${SCAFFOLDS}; do
  SAMPLE_NAME=$(echo "${SCAFFOLD}" | sed "s/.*\(ERR[0-9]\+\)\/output.*/\1/");
  OUTPUT_DIR="${WORKDIR}/outputs/antismash/${SAMPLE_NAME}";
  "${REPO}"/run_scripts/run_antismash.sh "${SCAFFOLD}" "${OUTPUT_DIR}" \
  --cpus 40 \
  --genefinding-tool prodigal-m \
  --taxon bacteria \
  --allow-long-headers \
  --minlength 5000;
done

***
#### **2) Taxonomic annotation of BGCs**
***

For this we will use out script [run_mmseqs_taxonomy.sh](https://github.com/pereiramemo/bioprospecting/blob/main/run_scripts/run_mmseqs_taxonomy.sh), which runs a container of [MMseqs](https://github.com/soedinglab/MMseqs2), utilizing [UniProtKB/Swiss-Prot](https://www.uniprot.org/uniprotkb?facets=reviewed%3Atrue&query=%2A) as the refrence database.

Before doing anything, we have to organize the data and select the scaffolds in which a BGC was annotated:

In [ ]:
%%bash
cat "${WORKDIR}"/outputs/bgc_abund/ERR*.tsv | cut -f1 | sort | uniq > "${WORKDIR}"/outputs/bgc_taxa/ids.txt
cat "${WORKDIR}"/data/sola/ERR*/output/scaffolds.fasta >  "${WORKDIR}"/outputs/bgc_taxa/all.fasta
"${seqtk}" subseq \
"${WORKDIR}"/outputs/bgc_taxa/all.fasta \
"${WORKDIR}"/outputs/bgc_taxa/ids.txt > \
"${WORKDIR}"/outputs/bgc_taxa/bgc.fasta

Now we can perform the taxonomic annotation.

In [ ]:
%%bash

"${REPO}"/run_scripts/run_mmseqs_taxonomy.sh \
"${WORKDIR}"/outputs/bgc_taxa/bgc.fasta \
"${WORKDIR}"/outputs/bgc_taxa/bgc_taxa_annot \
--threads 40 \
--tax-lineage 1 \
-v 0

Lastly, we will format the taxonomic datatable to make it more suitable for downstream analyses.

In [ ]:
%%bash

gawk -v FS="\t" -v OFS="\t" '{
  seq_id = $1; tax_level = $3; tax_path = $9;
  sample = gensub(/__.*/,"","g", seq_id);
  print sample,seq_id,tax_level,tax_path
  }' "${WORKDIR}"/outputs/bgc_taxa/bgc_taxa_annot_lca.tsv > \
  "${WORKDIR}"/outputs/bgc_taxa/bgc_taxa_annot_lca_formatted.tsv

***
#### **3) BGC mapping**
***

Here we will assess how similiar are our BGC sequences found in the metagenomic samples in relation to knonw BGC sequences.  
For this anlaysis we will run the [BiG-SLICE](https://github.com/pereiramemo/bigslice) to map our BGCs against (prevoulsy constructed) GCF models of the [MIBiG database V3](https://mibig.secondarymetabolites.org/).

To be able to run [BiG-SLICE](https://github.com/pereiramemo/bigslice) we have to format the input properly, that is, creating the [dataset.tsv and taxonomy files](https://github.com/medema-group/bigslice/wiki/Input-folder). 

In [ ]:
%%bash

ls -d "${WORKDIR}/outputs/antismash/"ERR* | \
while read LINE; do
  DATASET=$(basename $(ls -d ${LINE}))
  PATH2DATASET=$(basename $(dirname ${LINE}))"/"
  echo -e "${DATASET}\t./\ttaxonomy/${DATASET}_taxonomy.tsv\tdataset_${DATASET}"
done > "${WORKDIR}/outputs/antismash/datasets.tsv"

ls "${WORKDIR}/outputs/antismash/"ERR*/scaffolds/ERR*region*.gbk | \
while read LINE; do
  DATASET=$(basename ${LINE/__k*.region*.gbk//})
  SEQID=$(basename ${LINE/.region*.gbk//})
  OUTPUT_FILE="${WORKDIR}/outputs/antismash/taxonomy/${DATASET}_taxonomy.tsv"
  echo -e "${SEQID}\tBacteria" >> "${OUTPUT_FILE}"
done

Now we do the mapping.

In [ ]:
%%bash

"${REPO}/run_scripts/run_bigslice.sh" query \
"${WORKDIR}/outputs/antismash/" \
"${WORKDIR}/data/mibig_gcf_models/" \
--num_threads 40 \
--threshold_pct 0.1 \
--query_name SOLA 

***
#### **4) BGC clustering**
***

Here again will use [BiG-SLICE](https://github.com/medema-group/bigslice), this time to cluster our BGC sequences into Gene Cluster Families (GCFs).  

In [ ]:
%%bash

"${REPO}/run_scripts/run_bigslice.sh" cluster \
"${WORKDIR}/outputs/antismash/" \
"${WORKDIR}/outputs/bgslice_clust/" \
--num_threads 40 \
--threshold_pct 0.1 1> /dev/null 2> /dev/null

***
#### **5) Compute coverage**
***

To do this, let's we will first concatenat all BGC gbk files from each metagenomic sample.  

In [67]:
%%bash

ls -d ${WORKDIR}/outputs/antismash/ERR* | \
while read LINE; do
  SAMPLE=$(basename "${LINE}")
  cat "${LINE}"/scaffolds/"${SAMPLE}"*.region*.gbk > "${WORKDIR}"/outputs/bgc_abund/"${SAMPLE}".gbk
done

Now we can run our custom scripts [get_cov.py](https://github.com/pereiramemo/bioprospecting/blob/main/aux_scripts/get_cov.py) to compute the coverage of each BGC.

In [72]:
%%bash

ls "${WORKDIR}"/outputs/bgc_abund/*.gbk | \
while read LINE; do
  SAMPLE=$(basename "${LINE}" .gbk)
  "${REPO}"/aux_scripts/get_cov.py \
  --input_gbk "${WORKDIR}"/outputs/bgc_abund/"${SAMPLE}".gbk \
  --input_bam "${WORKDIR}"/data/sola/"${SAMPLE}"/output/mapped.sorted.bam \
  --sample_name "${SAMPLE}" \
  --output_tsv "${WORKDIR}"/outputs/bgc_abund/"${SAMPLE}".tsv
done

cat "${WORKDIR}"/outputs/bgc_abund/ERR*.tsv  > "${WORKDIR}"/outputs/bgc_abund/bgc_abund.tsv

***
#### **Format outputs**
***

Lastly, after we run the five main tasks that comprise the bioprospecting pipeline, we format the output data generated to obtain the three tables described in Fig. 1.  

#### **Output table 1**.  
That is, the table containing the functional and taxonomic annotation, and the biosynthetic novelty.  

In [85]:
%%R
library(RSQLite)
library(tidyverse)

TAX <- read_tsv("workdir/outputs/bgc_taxa/bgc_taxa_annot_lca_formatted.tsv", 
                col_names = F, show_col_types = F)
colnames(TAX) <- c("sample", "seq_id", "tax_level", "tax_path")
CONN_REPORTS_DB <- dbConnect(SQLite(), "workdir/data/mibig_gcf_models/reports/1/data.db")
CONN_DATA_DB <- dbConnect(SQLite(), "workdir/data/mibig_gcf_models/result/data.db")

REPORTS_LIST <- lapply(setNames(nm = dbListTables(CONN_REPORTS_DB)), dbReadTable, conn = CONN_REPORTS_DB)
DATA_LIST <- lapply(setNames(nm = dbListTables(CONN_DATA_DB)), dbReadTable, conn = CONN_DATA_DB)

REPORTS_LIST$bgc$seq_id <- sub(x = REPORTS_LIST$bgc$name, 
                               pattern = ".*/(.*).region[0-9]+$", 
                               replacement = "\\1")

metadata <- left_join(x = TAX, 
                      y = REPORTS_LIST$bgc,
                      by = "seq_id") %>%
            left_join(x = .,
                      y = REPORTS_LIST$gcf_membership, 
                      by = c("id" = "bgc_id")) %>%
            left_join(x = .,
                      y = REPORTS_LIST$bgc_class, 
                      by = c("id" = "bgc_id")) %>%
            left_join(x = .,
                      y = DATA_LIST$chem_subclass_map,
                      by = c("chem_subclass_id" = "subclass_id")) %>%
            select(seq_id, class_source, tax_path, membership_value, gcf_id)
        
colnames(metadata) <- c("seq_id", "function", "taxonomy", "novelty", "gcf_id")       
        
write.table(x = metadata, file = "workdir/outputs/tables/1-bgc_metadata.tsv", 
            sep = "\t", quote = F, row.names = F, col.names = T)

This is how it looks like:

In [86]:
%%bash

head "${WORKDIR}/outputs/tables/1-bgc_metadata.tsv"

seq_id	function	taxonomy	novelty	gcf_id
ERR2604090__k119_185408	terpene	-_cellular organisms;d_Bacteria;-_Terrabacteria group;-_Cyanobacteriota/Melainabacteria group;p_Cyanobacteriota;c_Cyanophyceae;o_Synechococcales	1.10505376873434	541
ERR2604090__k119_153928	arylpolyene	-_cellular organisms;d_Bacteria	0.952125515265246	1026
ERR2604090__k119_123795	terpene	-_cellular organisms;d_Bacteria;p_Pseudomonadota;c_Alphaproteobacteria;o_Rhodobacterales;f_Paracoccaceae	0.986966759230745	241
ERR2604090__k119_131898	redox-cofactor	-_cellular organisms;d_Bacteria;p_Pseudomonadota;c_Gammaproteobacteria	1.0320650538974	454
ERR2604090__k119_155472	redox-cofactor	-_cellular organisms;d_Bacteria;p_Pseudomonadota	1.12065717045078	454
ERR2604090__k119_52769	terpene	-_cellular organisms	1.26869265952904	391
ERR2604090__k119_148185	terpene	-_cellular organisms;d_Bacteria	1.14012573711592	391
ERR2604090__k119_84479	terpene	-_cellular organisms;d_Bacteria;p_Pseudomonadota	1.25337218528236	97
ERR2604090__k11

#### **Output table 2**  
That is, the BGC class abundance table.

In [75]:
%%R

BGC_TBL <- read_tsv("workdir/outputs/bgc_abund/bgc_abund.tsv", 
                    col_names = F,
                    show_col_types = F)
colnames(BGC_TBL) <- c("seq_id", "bgc_class", "edge", "start", "end", "abund", "sample")

BGC_TBL_coll <- BGC_TBL %>%
                group_by(sample, bgc_class) %>%
                summarize(abund = sum(abund))
    
write.table(x = BGC_TBL_coll, file = "workdir/outputs/tables/2-bgc_abund.tsv", 
            sep = "\t", quote = F, row.names = F, col.names = T)

`summarise()` has grouped output by 'sample'. You can override using the
`.groups` argument.


This is how it looks like:

In [77]:
%%bash
head "${WORKDIR}/outputs/tables/2-bgc_abund.tsv"

sample	bgc_class	abund
ERR2604071	betalactone	1.01401006711409
ERR2604071	phosphonate	1.04183496826313
ERR2604071	resorcinol	1.01119288561791
ERR2604071	terpene	4.11969119492286
ERR2604073	betalactone	3.18565274708853
ERR2604073	hserlactone	1.06897456071335
ERR2604073	phosphonate	1.05259071729958
ERR2604073	redox-cofactor	1.07977006857604
ERR2604073	terpene	7.43873731742168


Note: For simplicity, we have created the tidy version of the table described in Fig. 1.

#### **Output table 3**
That is, the GCF abundance table.

In [ ]:
%%R 
